### 

In [104]:
import numpy as np
import pandas as pd
import re
from html.parser import HTMLParser
import requests
from bs4 import BeautifulSoup as bs
from html import unescape as ue
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [107]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/postas/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
train_df = pd.read_csv('train_tweets.csv')
test_df = pd.read_csv('test_tweets.csv')

In [3]:
train_df.shape

(31962, 3)

In [4]:
train_df.head(3)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty


In [5]:
test_df.head(3)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...


In [6]:
test_df.shape

(17197, 2)

In [7]:
df = test_df.merge(train_df)
df.shape

(0, 3)

In [8]:
test_df.shape

(17197, 2)

In [9]:
df = train_df.append(test_df)

In [10]:
df.shape

(49159, 3)

In [11]:
df

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation
...,...,...,...
17192,49155,NaN,thought factory: left-right polarisation! #tru...
17193,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
17194,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...
17195,49158,NaN,"happy, at work conference: right mindset leads..."


In [12]:
def unescape(string):
    return ue(string)
v_unescape = np.vectorize(unescape)

In [13]:
def subst_pattern(string, pattern, substitution=''):
    return re.sub(pattern, substitution, string)
v_subst_pattern = np.vectorize(subst_pattern)

In [14]:
def to_lower(string):
    return string.lower()
v_to_lower = np.vectorize(to_lower)

In [15]:
# Нашел в сети, практически готовый словарик

contractions = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [16]:
# Сокращения непонятно где в готовом виде взять. 
# Можно запарсить с https://www.webopedia.com/reference/text-abbreviations/

In [17]:
user_agent = {
        'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
url = 'https://www.webopedia.com/reference/text-abbreviations/'
req = requests.get(url, headers=user_agent)
if req.ok:
    html = req.text
    parsed_html = bs(html, 'lxml')

In [18]:
abbreviations_table = parsed_html.find('table', {'class': 'style5'})

In [19]:
abbreviation_pairs = [pair for pair in abbreviations_table.findAll('tr') if len(pair('td')) > 1]

In [20]:
abbreviations = dict([[cell.text.lower() for cell in row('td')] for row in abbreviation_pairs])

In [21]:
abbreviations

{'?': 'i don’t understand what you mean',
 '?4u': 'i have a question for you',
 ';s': 'gentle warning, like “hmm? what did you say?”',
 '^^': 'meaning “read line” or “read message” above',
 '': 'meaning “heart or love” (more 3s is a bigger heart)',
 '@teotd': 'at the end of the day',
 '.02': 'my (or your) two cents worth',
 '1tg, 2tg': 'meaning number of items needed for win (online gaming)',
 '1up': 'meaning extra life (online gaming)',
 '121': 'one-to-one (private chat initiation)',
 '1337': 'leet, meaning ‘elite’',
 '143': 'i love you',
 '1432': 'i love you too',
 '14aa41': 'one for all, and all for one',
 '182': 'i hate you',
 '19': 'zero hand (online gaming)',
 '10m': 'ten man (online gaming)',
 '10x': 'thanks',
 '10q': 'thank you',
 '1ce': 'once',
 '1dr': 'i wonder',
 '1nam': 'one in a million',
 '2': 'meaning “to” in sms',
 '20': 'meaning “location”',
 '2b': 'to be',
 '2ez': 'too easy',
 '2g2bt': 'too good to be true',
 '2m2h': 'too much to handle',
 '2mi': 'too much information

In [64]:
# Вопросительный знак я бы убрал точно из этого словаря )
abbreviations.pop('?')
abbreviations.pop('')

'meaning “heart or love” (more 3s is a bigger heart)'

In [23]:
# Эмотиконы
# Тот-же источник https://www.webopedia.com/reference/smiley-face-emoji-meaning/

In [24]:
url = 'https://www.webopedia.com/reference/smiley-face-emoji-meaning/'
req = requests.get(url, headers=user_agent)
if req.ok:
    html = req.text
    parsed_html = bs(html, 'lxml')

In [25]:
emoticons_table = parsed_html.find('table', {'class': 'auto-style1'})

In [26]:
emoticon_pairs = [pair for pair in emoticons_table.findAll('tr') if len(pair('td')) > 1]

In [27]:
emoticons = dict([[cell.text.strip('\n').lower() for cell in row('td')] for row in emoticon_pairs][1:])

In [28]:
emoticons

{'🙂': 'with nose',
 ':-e': 'buck-tooth or vampire',
 '>-)': 'evil grin',
 '🙁': 'sad with nose',
 ':-': 'shifty',
 '😛': 'sticking tongue out (raspberry)',
 ':-o': 'surprised',
 ':-*': 'kissing',
 ':[email\xa0protected] ': 'angry',
 ':-$': 'confused',
 ':-#': 'with braces',
 '(((h)))': 'hugs',
 ':-x': 'kiss on the lips',
 '`:-)': 'one eyebrow raised',
 ':^)': 'a broken nose',
 ':-&': 'tongue tied',
 'e-:-)': 'a ham radio operator',
 '': 'robot',
 ':->': 'big grin happy',
 '(-}{-)': 'couple kissing',
 ':-q': 'smoking',
 '$_$': 'greedy',
 '@@': 'rolling your eyes',
 ':-!': '“foot in mouth”',
 '😀': 'laughter',
 ':*)': 'drunk smile',
 ':@': 'exclamation “what???”',
 ':[email\xa0protected]': 'scream',
 ':-0': 'yell',
 ':—–)': 'long nose (liar!)',
 '%-(': 'confused',
 ':-.)': 'madonna',
 ':-($)': 'put your money where your mouth is',
 '(:i': 'an egghead',
 '|-o': 'yawn',
 ':@)': 'pig smile',
 'd[-_-]b': 'dj with headphones',
 '~:0': 'baby',
 '[email\xa0protected]–@-': 'eyeglasses',
 'vvv/': 'k

In [62]:
# Небольшие дополнения...
emoticons[':-)'] = 'smile'
emoticons[':)'] = 'happy'
emoticons.pop('')

'robot'

In [59]:
# Функция для замены по словарям
def subs_with_LUT(string, LUT):
    return ' '.join([LUT[word] if word in LUT else word for word in re.split(r'\W', string)])
v_subs_with_LUT = np.vectorize(subs_with_LUT)

In [60]:
# Однобуквенные слова
def one_char_word(string):
    return ' '.join([word for word in re.split(r'\W', string) if len(word) > 1])
v_one_char_word = np.vectorize(one_char_word)

### Очистка

In [65]:
df['clean_tweet'] = v_unescape(df.tweet)
df['clean_tweet'] = v_to_lower(df.clean_tweet)
df['clean_tweet'] = v_subst_pattern(df.clean_tweet, r'@[\w]*', '')
df['clean_tweet'] = v_subs_with_LUT(df.clean_tweet, {**contractions, **abbreviations, **emoticons})
df['clean_tweet'] = v_subst_pattern(df.clean_tweet, r'[^\w\s]', ' ')
df['clean_tweet'] = v_subst_pattern(df.clean_tweet, r'[^a-zA-Z0-9]', ' ')
df['clean_tweet'] = v_subst_pattern(df.clean_tweet, r'[^a-zA-Z]', ' ')
df['clean_tweet'] = v_one_char_word(df.clean_tweet)
df

,id,label,tweet,clean_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is significan...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit can use cause they don ...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,model love take with all the time in ur
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation
...,...,...,...,...
17192,49155,NaN,thought factory: left-right polarisation! #tru...,thought factory left right polarisation trump ...
17193,49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like mermaid hairflip neverready forma...
17194,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...,hillary campaigned today in ohio oh my gosh us...
17195,49158,NaN,"happy, at work conference: right mindset leads...",happy at your terminal work conference right m...


Немного громоздко, удобнее было бы это завернуть в sklearn.pipeline и пачкой применять трансформации к датасету

In [81]:
# Токенизация

In [83]:
df['tweet_token'] = df.clean_tweet.apply(lambda x: word_tokenize(x))
df.head()

,id,label,tweet,clean_tweet,tweet_token
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is significan...,"[when, father, is, dysfunctional, and, is, sig..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit can use cause they don ...,"[thanks, for, lyft, credit, can, use, cause, t..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]"


In [85]:
# Стоп-слова

In [88]:
stop_words = set(stopwords.words("english"))

In [95]:
def filter_stop_words(words):
    return [word for word in words if word not in stop_words]

In [98]:
df['tweet_token_filtered'] = df.tweet_token.apply(lambda x: filter_stop_words(x))
df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is significan...,"[when, father, is, dysfunctional, and, is, sig...","[father, dysfunctional, significant, selfish, ..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit can use cause they don ...,"[thanks, for, lyft, credit, can, use, cause, t...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]"


In [ ]:
# Stemming

In [101]:
def stemming(words, stemmer = PorterStemmer()):
    return [stemmer.stem(word) for word in words]

In [102]:
df['tweet_stemmed'] = df.tweet_token_filtered.apply(lambda x: stemming(x))
df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is significan...,"[when, father, is, dysfunctional, and, is, sig...","[father, dysfunctional, significant, selfish, ...","[father, dysfunct, signific, selfish, drag, ki..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit can use cause they don ...,"[thanks, for, lyft, credit, can, use, cause, t...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]"
3,4,0.0,#model i love u take with u all the time in ...,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]"


In [103]:
# Lemmatizing

In [105]:
def lemmatizing(words, lemmatizer=WordNetLemmatizer()):
    return [lemmatizer.lemmatize(word) for word in words]

In [108]:
df['tweet_lemmatized'] = df.tweet_token_filtered.apply(lambda x: lemmatizing(x))
df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is significan...,"[when, father, is, dysfunctional, and, is, sig...","[father, dysfunctional, significant, selfish, ...","[father, dysfunct, signific, selfish, drag, ki...","[father, dysfunctional, significant, selfish, ..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit can use cause they don ...,"[thanks, for, lyft, credit, can, use, cause, t...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love take with all the time in ur,"[model, love, take, with, all, the, time, in, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [111]:
df[['tweet_token_filtered', 'tweet_lemmatized']]

,tweet_token_filtered,tweet_lemmatized
0,"[father, dysfunctional, significant, selfish, ...","[father, dysfunctional, significant, selfish, ..."
1,"[thanks, lyft, credit, use, cause, offer, whee...","[thanks, lyft, credit, use, cause, offer, whee..."
2,"[bihday, majesty]","[bihday, majesty]"
3,"[model, love, take, time, ur]","[model, love, take, time, ur]"
4,"[factsguide, society, motivation]","[factsguide, society, motivation]"
...,...,...
17192,"[thought, factory, left, right, polarisation, ...","[thought, factory, left, right, polarisation, ..."
17193,"[feeling, like, mermaid, hairflip, neverready,...","[feeling, like, mermaid, hairflip, neverready,..."
17194,"[hillary, campaigned, today, ohio, oh, gosh, u...","[hillary, campaigned, today, ohio, oh, gosh, u..."
17195,"[happy, terminal, work, conference, right, min...","[happy, terminal, work, conference, right, min..."


In [112]:
# Что-то лемматизация от исходника не особо отличается в английском языке :))

In [113]:
# Аутпут
df.to_pickle('./tweets_preprocessed.pkl')